In [2]:
!pip install scikit-learn


   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.7 MB 7.0 MB/s eta 0:00:02
   -------- ------------------------------- 1.8/8.7 MB 5.0 MB/s eta 0:00:02
   ------------ --------------------------- 2.6/8.7 MB 4.6 MB/s eta 0:00:02
   --------------- ------------------------ 3.4/8.7 MB 4.1 MB/s eta 0:00:02
   -------------------- ------------------- 4.5/8.7 MB 4.3 MB/s eta 0:00:02
   ------------------------ --------------- 5.2/8.7 MB 4.2 MB/s eta 0:00:01
   --------------------------- ------------ 6.0/8.7 MB 4.1 MB/s eta 0:00:01
   ------------------------------- -------- 6.8/8.7 MB 4.1 MB/s eta 0:00:01
   ---------------------------------- ----- 7.6/8.7 MB 4.1 MB/s eta 0:00:01
   -------------------------------------- - 8.4/8.7 MB 4.1 MB/s eta 0:00:01
   ---------------------------------------- 8.7/8.7 MB 4.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.4 MB ? eta -:--:--
    ---------------------

Importing Libraries

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

Data Collection

In [6]:
df = pd.read_csv("ifood_df.csv")  # Replace with actual file name
print("Data Loaded Successfully!")
print(df.head())


Data Loaded Successfully!
    Income  Kidhome  Teenhome  Recency  MntWines  MntFruits  MntMeatProducts  \
0  58138.0        0         0       58       635         88              546   
1  46344.0        1         1       38        11          1                6   
2  71613.0        0         0       26       426         49              127   
3  26646.0        1         0       26        11          4               20   
4  58293.0        1         0       94       173         43              118   

   MntFishProducts  MntSweetProducts  MntGoldProds  ...  marital_Together  \
0              172                88            88  ...                 0   
1                2                 1             6  ...                 0   
2              111                21            42  ...                 1   
3               10                 3             5  ...                 1   
4               46                27            15  ...                 0   

   marital_Widow  education_2n

Data Exploration

In [7]:
print(df.info())
print(df.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2205 entries, 0 to 2204
Data columns (total 39 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Income                2205 non-null   float64
 1   Kidhome               2205 non-null   int64  
 2   Teenhome              2205 non-null   int64  
 3   Recency               2205 non-null   int64  
 4   MntWines              2205 non-null   int64  
 5   MntFruits             2205 non-null   int64  
 6   MntMeatProducts       2205 non-null   int64  
 7   MntFishProducts       2205 non-null   int64  
 8   MntSweetProducts      2205 non-null   int64  
 9   MntGoldProds          2205 non-null   int64  
 10  NumDealsPurchases     2205 non-null   int64  
 11  NumWebPurchases       2205 non-null   int64  
 12  NumCatalogPurchases   2205 non-null   int64  
 13  NumStorePurchases     2205 non-null   int64  
 14  NumWebVisitsMonth     2205 non-null   int64  
 15  AcceptedCmp3         

Data Cleaning

In [9]:
df.drop_duplicates(inplace=True)
df.dropna(inplace=True)

RFM

In [10]:
snapshot_date = df['InvoiceDate'].max()
rfm = df.groupby('CustomerID').agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'InvoiceNo': 'count',
    'Amount': 'sum'
})

rfm.columns = ['Recency', 'Frequency', 'Monetary']
rfm = rfm.reset_index()

KeyError: 'InvoiceDate'